##  基于词典匹配的实体抽取方法


本例子配合 《知识图谱：认知智能理论与实战》3.2.1节《基于词典匹配的实体抽取方法》使用。

使用Trie树数据结构（也称字典树、前缀树）来构建词典，并通过前向最大匹配，从一串文本中找到词典中的实体。基于词典匹配的方法在深度学习兴起以前广泛应用在搜索引擎、分词和实体抽取中。即使在今天，许多场景下使用词典匹配的方法也能够很好地满足实体抽取的业务需求。

参考内容： 
- 树结构说明： 《知识图谱：认知智能理论与实战》3.2.1节《基于词典匹配的实体抽取方法》，P83-84
- Trie 树例子： 《知识图谱：认知智能理论与实战》图3-1，P83-84
- wikipedia Trie 词条： [英文](https://en.wikipedia.org/wiki/Trie) [中文](https://zh.wikipedia.org/wiki/Trie)


## Trie 树数据结构及正向最大匹配

In [1]:
class TrieNode:
    """Trie 树的节点"""
    def __init__(self, c):
        #  保存字符
        self.c = c
        # 是否是一个词的终止字
        self.is_end = False
        # 保存子节点，key 为字，value 为节点
        self.children = {}
        
class Trie(object):
    """Trie 树"""
    def __init__(self):
        """根节点，空的，不保存任何字符"""
        self.root = TrieNode('\x01')
    
    def add(self, w):
        """插入词，构建 Trie 树"""
        n = self.root
        # 插入到合适的节点中
        for c in w:
            if c in n.children:
                n = n.children[c]
            else:
                nn = TrieNode(c)
                n.children[c] = nn
                n = nn
        # 词的最后一个字所对应的节点
        n.is_end = True
    
    def fmm(self, s):
        """正向最大匹配(forward maximum matching)算法"""
        results = []
        slen = len(s)
        i = 0
        while i < slen:
            n = self.root
            w = ''
            for j in range(i, slen):
                c = s[j]
                if c in n.children:
                    n = n.children[c]
                    w += c
                else:
                    if n.is_end:
                        # 如果匹配出一个词，则从该词的结束位置开始下一次匹配
                        results.append(w)
                        i = j-1
                    break
            i += 1
        return results



## 载入中国省市县数据，构建 Trie 树

In [2]:
t = Trie()
with open('省市县词典.txt') as f:
    for line in f:
        t.add(line.strip())


## 利用最大前向匹配抽取实体

从百度搜索“联系地址 site:gov.cn”，可以看到大量的地址信息，可用于测试。



In [3]:
s = '地址信息 地址:广东省深圳市龙岗区龙翔大道8033号 邮政编码:518100 电子邮箱:lgxfj@lg.gov.cn 技术支持:龙岗区政务服务数据管理局'
s = '四川省崇州监狱 加入收藏设为首页 2022年8月9日星期二 您现在所在的位置首页>四川省崇州监狱>便民服务>联系地址 四川省崇州监狱 四川省崇州市崇庆街道西桥路242号'
s = '广东省深圳市龙岗区龙翔大道8033号'
s = '办公地址:北京市西城区文兴东街1号国谊宾馆(过渡期办公) | 联系我们 邮政地址:北京市海淀区复兴路乙15号 | 邮政编码:100862 ICP备案序号:京ICP备05022684 | 网站标识码:bm060...'
s = '信息来源: 发布时间:2020-04-26 13:28【打印】【关闭】 联系电话:028-88522571 联系地址:成都市蒲江县鹤山镇成祥街1号 邮编:611630 分享到0'
s = '序号单位名称详细地址联系电话 1市公安局办公室枣庄薛城区光明大道2277号0632-3656010 2市局民生警务中心枣庄薛城区光明大道2277号9600110 3市局户政管理处枣庄新城长白山南...'
s = '四川省川中监狱 联系地址:四川省南充市高坪区鹤鸣路548号 乘车提示:嘉陵汽车站可乘3路、14路公交车,在南充市顺庆区五星花园下车,到模范街转乘2路公交车至其终点站下车即到;城...'
s = '办公及通信地址:广州市天河区珠江新城花城大道83号、66号,邮编:510623 业务咨询电话:020-12360转1(广州海关) 廉政举报电话:020-81103621 走私举报电话:020-81102105、8759109...'

t.fmm(s)


['广州市', '天河区']